In [ ]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [907 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.

In [ ]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-01 01:22:14--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.62MB/s    in 0.2s    

2023-03-01 01:22:14 (5.62 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [ ]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [ ]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Outdoors_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   18446823|R35T75OLUGHL5C|B000NV6H94|     110804376|Stearns Youth Boa...|        Outdoors|          4|            0|          0|   N|                Y|          Four Stars|          GOOD VALUE|2015-01-31 00:08:00|
|         US|   13724367|R2BV735O46BN33|B000IN0W3Y|     624096774|Primal Wear Men's...| 

In [ ]:
# Get the number of rows in the DataFrame.
df.count()

2302401

In [ ]:
#Drop null values if any
df = df.dropna()
df.count()

2302174

# Transform the Data

In [ ]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [ ]:
# Change datatypes
# Source Link: https://stackoverflow.com/questions/32284620/how-to-change-a-dataframe-column-from-string-type-to-double-type-in-pyspark

from pyspark.sql.types import TimestampType, IntegerType

df = df.withColumn("customer_id", df["customer_id"].cast(IntegerType()))\
       .withColumn("product_parent", df["product_parent"].cast(IntegerType()))\
       .withColumn("review_date", df["review_date"].cast(TimestampType()))\
       .withColumn("star_rating", df["star_rating"].cast(IntegerType()))\
       .withColumn("helpful_votes", df["helpful_votes"].cast(IntegerType()))\
      .withColumn("total_votes", df["total_votes"].cast(IntegerType()))\

# Print schema
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



## Create the "review_id_table".

In [ ]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = df.select(["review_id","customer_id","product_id", "product_parent","review_date"])
review_id_df.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R35T75OLUGHL5C|   18446823|B000NV6H94|     110804376|2015-01-31 00:08:00|
|R2BV735O46BN33|   13724367|B000IN0W3Y|     624096774|2015-01-31 00:08:00|
|R2NBEUGPQQGXP1|   51001958|B008RBJXFM|     278970944|2015-01-31 00:08:00|
|R17LLAOJ8ITK0S|   32866903|B00FK8WUQY|     312877650|2015-01-31 00:08:00|
|R39PEQBT5ISEF4|   30907790|B00EZA3VW0|     305567912|2015-01-31 00:08:00|
|R3GNM3SU9VHJFT|   20232229|B006JA8WEG|     842306035|2015-01-31 00:08:00|
| R2Y81OP0EK467|   17698862|B002PWFSEO|     451480122|2015-01-31 00:08:00|
|R2LFGSI6HAYH5F|   38486114|B002DZGKHW|     124386306|2015-01-31 00:08:00|
|R297G6ED1IQO7W|   26319572|B00ABA08F6|     991442421|2015-01-31 00:08:00|
| RE27RFC6101N6|   27152337|B003Z8WIHC|     886483892|2015-01-31 00:08:00|
|R3BPDME6E94W8Z|   125168

In [ ]:
# Print schema
review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: timestamp (nullable = true)



## Create the "products" Table

In [ ]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = df.select(["product_id","product_title"])
products_df = products_df.dropDuplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00IFHFJXI|Ivation Portable ...|
|B00WG0J0D0|JanSport Superbre...|
|B00V15AUN0|Nickelodeon Paw P...|
|B00FUWSTI8|Bago Lightweight ...|
|B003FV94NA|Michelin Lithion ...|
|B00WIK04HO|Ultra Bright Camp...|
|B00J2HSCM0|High Sierra Tank ...|
|B009I6NSR4|Black Veil Brides...|
|B001GSHSLE|Stansport 191 App...|
|B00L2IO9M4|Columbia Sportswe...|
|B00KY7IM7W|Nalgene 32 Oz Wid...|
|B00TV5JCTK|Rollerblade ABEC ...|
|B00B9D071Y|BUFF UV Multifunc...|
|B00F9IGIKO|Condor Tactical F...|
|B004X55L9I|Hydro Flask Insul...|
|B00LORROIY|Scuba Choice Divi...|
|B00AATRU8G|Kelty Redwing 44 ...|
|B00HMCYWEO|Dakine Explorer L...|
|B004DK1CM8|Hot Headz 12V Hea...|
|B00T4W6SSS|Fits Sock Light H...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [ ]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
from pyspark.sql.functions import desc

customers_df = df.groupby("customer_id").agg({"customer_id":"count"})
customers_df = customers_df.orderBy(desc("count(customer_id)"))
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count") 
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   13355404|           291|
|   47355039|           205|
|   51155788|           197|
|   20433453|           129|
|   40944345|           128|
|    3247563|           128|
|   27550730|           125|
|   30636778|           115|
|   10796373|           113|
|   40364490|           112|
|   16255502|           112|
|   38137849|           111|
|   52988261|           109|
|   36205928|           108|
|   52196389|           104|
|   18466687|           104|
|   44142946|           102|
|   49782074|           100|
|   45014035|           100|
|   31807575|            98|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [ ]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
# Load in a sql function to use columns
from pyspark.sql.functions import col

vine_table_df = df.select(["review_id","star_rating","helpful_votes", "total_votes","vine"])

# # Filter for only columns from Amazon's Vine program
# vine_table_df = vine_table_df.filter(col("vine")  == "Y")
vine_table_df.show() 

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R35T75OLUGHL5C|          4|            0|          0|   N|
|R2BV735O46BN33|          5|            0|          0|   N|
|R2NBEUGPQQGXP1|          4|            0|          0|   N|
|R17LLAOJ8ITK0S|          3|            1|          1|   N|
|R39PEQBT5ISEF4|          1|            0|          0|   N|
|R3GNM3SU9VHJFT|          4|            1|          1|   N|
| R2Y81OP0EK467|          5|            0|          0|   N|
|R2LFGSI6HAYH5F|          5|            1|          1|   N|
|R297G6ED1IQO7W|          5|            1|          1|   N|
| RE27RFC6101N6|          5|            0|          0|   N|
|R3BPDME6E94W8Z|          5|            0|          0|   N|
|R2P08O1RILUOX3|          3|            0|          0|   N|
|R37CVAB03PTDVI|          5|            0|          1|   N|
| RAWNWOGXPCPMD|          5|            

# Load

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://database-1.cwtdgnuu2aym.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", "password": "Postgres", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)